Given a $m \times n$ matrix $I$ with each row being a $n$-dimensional vector, we are to output an "importance" matrix $O$ where the value at $O_{i,j}$ is a measure of how different value $I_{i,j}$ is from the rest of the values in row $i$ with $0 \leq O_{i,j} \leq 1$. This matrix must satisfy two conditions- first, that if all values in a column in $I$ were the same, the values in the same column in $O$ should be $0$. Second, if two values are different out of a larger set of values that are the same, the corresponding values to the two different values in $O$ should be $1$ with the rest same being $0$. 

Any solution will be focused on the importance function, which should take in a $1 \times m$ vector and return the importance of each value in the form of a  $1 \times m$ vector. We're not told if the underlying data is continuous or discrete, so we'll implement importance functions for both and allow the user to choose which to use.  Due to this being a python project working on matrices, it should input and output a numpy matrix. I want the user to be able to adjust the importance function and still output a matrix, so we'll both allow the user to input their own importance function to apply to the columns. 

For categorical data, I have a very obvious implementation of the importance function. I will make the importance function inversely proportional the count of a value, then scale these numbers between 0 and 1. I can make use of the threshold to include values with lower distances than the threshold together to allow this to work well for continuous values as well- this function is defined as 

$count\_within\_threshold\_importance(array, t)[i] \propto \frac{1}{len(x | abs(x- array[i]) < t)}$ with 

$max(count\_within\_threshold\_importance(array,t)) = 1$ and $min(count\_within\_threshold\_importance(array,t)) = 0$ 

If array has no distinct values, the function returns 0. This function allows rarer values to have higher importance but adjust for the effects of noise, and should be used for categorical data, discrete data, and data with known noise-amount. 

In [1]:
from importance_matrix import ImportanceMatrix
import numpy as np

print(ImportanceMatrix.count_within_threshold_importance(np.array([1,1,1,1,1.1,2,3])))
print(ImportanceMatrix.count_within_threshold_importance(np.array([1,1,1,1,1.1,2,3]), threshold=.11))

[0. 0. 0. 0. 1. 1. 1.]
[0. 0. 0. 0. 0. 1. 1.]


For multimodal and continuous data, I provided a function based on clustering values together, and ranking values as having higher importance if they belong to a smaller cluster and have a high z-score within their cluster. This was chosen because values within small clusters should be considered more important, and values further away from the mean of a cluster should also be considered more important.

The clustering function needed an easy to be resistant to changes in array order and should require only a easily interpretable and statistically relevant threshold to operate. I wrote a custom algorithm to do this (though I am sure the method has been used before). Let the array be $a$, $\mu_x$ be the mean of array $x$, $\sigma_x$ the standard deviation of $x$, and $p$ the threshold chosen. Nearby values were first grouped into clusters ("nearbye" defined as having a distance less than $j=\frac{\sigma_a(1-p)}{2}$). Then, each cluster was compared each other cluster by performing a two-tailed student's t-test. If the t-test returned a $p$ value greater than $p$, the clusters were combined. This process was repeated until no more clusters were combined. Scores were then proportional to the inverse of the size of cluster a value belonged to plus the absolute value of the z-score times $k=\frac{1}{m(m+1)}$ where $m$ is the size of the largest cluster. 

$p$ can be interpreted as the minumum random chance of two clusters, intererpeted as being sampled from normal distributions, belonging to the same population at which we interpret the two populations as seperate. **An increased $p$ decreases the chance that clusters are combined.** $j$ was chosen by trialing different test inputs, with the eye on that $j$ should decrease with $p$. $k$ was multiplied to the z-score of a value within its cluster to stop any value from a larger cluster from having a higher importance than any value in a smaller cluster unless it had a very high z-score. 

In [2]:
from importance_matrix import ImportanceMatrix
import numpy as np

print(ImportanceMatrix.gaussian_clusters_importance(np.array([1,1,1,1,1,18,-18,1])))
print(ImportanceMatrix.gaussian_clusters_importance(np.array([1,1,1,1,1.1,2,3]), threshold=.10))
print(ImportanceMatrix.gaussian_clusters_importance(np.array([1,1,1,1,1.1,2,3]), threshold=.90))

[0. 0. 0. 0. 0. 1. 1. 0.]
[0.00757576 0.00757576 0.00757576 0.00757576 0.         0.05555556
 1.        ]
[0. 0. 0. 0. 1. 1. 1.]


To use the package, use pip install -r requirements.txt. In your code, import ImportanceMatrix from importance_matrix.
The constructor of importance matrix takes your input matrix data as a 2-D numpy ndarray, your threshold, and your importance function, which can be ```"count_within_threshold"``` for ```count_within_threshold_importance```,  ```"normal_dist_clustering" ``` for ```gaussian_clusters_importance```, or your own function which inputs a 1-D array and a threshold and returns a 1-D array. The attribute importance holds a 2-D ```ndarray``` containing the output of the importance function

In [4]:
from importance_matrix import ImportanceMatrix
import numpy as np

test_val_1 = np.array([[15,16,17], [15,16,17], [15,16,17], [15,16,17]])
test_val_2 = np.array([[15,0,17], [15,16,17], [15,16,17], [15,16,17]])
test_val_3 = np.array([[1],[1],[1],[1],[1.1],[2],[3]])
test_val_4 = np.array([[1,15],[1,15],[1,2],[1,20],[1.1,15],[2,15],[3,15]])

print("Using count_within_threshold")
print(ImportanceMatrix(test_val_1, 0, "count_within_threshold"))
print(ImportanceMatrix(test_val_2, 0, "count_within_threshold"))
print(ImportanceMatrix(test_val_3, .11, "count_within_threshold"))
print(ImportanceMatrix(test_val_4, 0, "count_within_threshold"))
print("Using normal_dist_clustering")
print(ImportanceMatrix(test_val_1, 0.05, "normal_dist_clustering"))
print(ImportanceMatrix(test_val_2, 0.1, "normal_dist_clustering"))
print(ImportanceMatrix(test_val_3, .9, "normal_dist_clustering"))
print(ImportanceMatrix(test_val_4, .1, "normal_dist_clustering"))


out = ImportanceMatrix(test_val_4, .1, "normal_dist_clustering")
print(type(out))
print(out[0,0])
print(type(out.importance))

Using count_within_threshold
ImportanceMatrix: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
ImportanceMatrix: [[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
ImportanceMatrix: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]]
ImportanceMatrix: [[0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Using normal_dist_clustering
ImportanceMatrix: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
ImportanceMatrix: [[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
ImportanceMatrix: [[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]
ImportanceMatrix: [[0.00757576 0.        ]
 [0.00757576 0.        ]
 [0.00757576 1.        ]
 [0.00757576 0.05555556]
 [0.         0.        ]
 [0.05555556 0.        ]
 [1.         0.        ]]
<class 'importance_matrix.importance.ImportanceMatrix'>
0.007575757575757597
<class 'numpy.ndarray'>


c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
